In [1]:
import numpy as np
import scipy.linalg as sal

In [2]:
def inner_product(left_vec, right_vec):
    bra = (np.conj(left_vec)).T
    ket = right_vec
    inner_prod = bra @ ket
    return inner_prod

In [3]:
def norm_column_vec(x):
    # m, n = x.shape
    inner_pro = inner_product(x, x)
    # inner_prodcut is still a 1x1 array, lets extract the element and take its sqrt
    norm = np.sqrt(np.squeeze(inner_pro))
    x_norm = x / norm
    return x_norm

In [4]:
def individual_spin_operation_matrix():
    S1_2 = np.array([[3/4,0,0,0],[0,3/4,0,0],[0,0,3/4,0], [0,0,0,3/4]])
    S1z = np.array([[1/2,0,0,0],[0,1/2,0,0],[0,0,-1/2,0], [0,0,0,-1/2]])
    S2z = np.array([[1/2,0,0,0],[0,-1/2,0,0],[0,0,1/2,0], [0,0,0,-1/2]])
    S1_plus = np.array([[0,0,1,0],[0,0,0,1],[0,0,0,0], [0,0,0,0]])
    S2_plus = np.array([[0,1,0,0],[0,0,0,0],[0,0,0,1], [0,0,0,0]])
    S1_minus = np.array([[0,0,0,0],[0,0,0,0],[1,0,0,0], [0,1,0,0]])
    S2_minus = np.array([[0,0,0,0],[1,0,0,0],[0,0,0,0], [0,0,1,0]])
    
    operate = {"S1_2": S1_2,
               "S2_2": S1_2,
               "S1z": S1z,
               "S2z" : S2z,
               "S1_plus": S1_plus,
               "S2_plus" : S2_plus,
               "S1_minus": S1_minus,
               "S2_minus" : S2_minus
              }
    return operate

In [5]:
def S_2_operate(function_list):
    l = len(function_list)
    S_2_mat = np.zeros((l,l))
    operate = individual_spin_operation_matrix()
    S1_2 = operate["S1_2"]
    S2_2 = operate["S2_2"]
    S1_plus = operate["S1_plus"]
    S2_plus = operate["S2_plus"]
    S1_minus = operate["S1_minus"]
    S2_minus = operate["S2_minus"]
    S1z = operate["S1z"]
    S2z = operate["S2z"]
    
    S_2_operation = S1_2 + S2_2 + 2*(0.5*(S1_plus@S2_minus+ S1_minus@S2_plus) + S1z@S2z)

    return S_2_operation

In [6]:
def Sz_operate(function_list):
    l = len(function_list)
    Sz_mat = np.zeros((l,l))
    operate = individual_spin_operation_matrix()
    S1z = operate["S1z"]
    S2z = operate["S2z"]
    
    Sz_operation = S1z + S2z
    
    return Sz_operation

In [7]:
def product_bases():
    operate = individual_spin_operation_matrix()
    S1_2 = operate["S1_2"]
    eig_val, eig_vec = sal.eig(S1_2)
    up_up = eig_vec[0].reshape(-1,1)
    up_down = eig_vec[1].reshape(-1,1)
    down_up = eig_vec[2].reshape(-1,1)
    down_down = eig_vec[3].reshape(-1,1)
    
    function_list = [up_up, up_down, down_up, down_down]
    function_dict = {"up_up": up_up,
                "up_down": up_down,
                "down_up": down_up,
                "down_down": down_down}
    return function_list, function_dict

In [8]:
def CG_coeff(function_list):
    S_2_mat = S_2_operate(function_list)
    coupled_val ,coupled_vec = sal.eig(S_2_mat)
    CG_matrix = coupled_vec.T
    
    return CG_matrix

In [9]:
def coupled_bases(CG_matrix):
    t1 = CG_matrix[2]
    t0 = CG_matrix[0]
    t_1 = CG_matrix[3]
    s0 = CG_matrix[1]
    coupled_states = {"triplet1": t1,
                  "triplet0": t0,
                  "triplet_1": t_1,
                  "singlet": s0
                 }
    return coupled_states

                      ## Magnetic Dipole-Dipole Interaction

    H = (mu1 . mu2)/r^3 - 3*(mu1 . r)*(mu2 . r)/r^5 ;
    since for an electron, mu = (g_e*e/(2*m_e))* S 
    for two electrons, the dipole_dipole interaction:
    H = (g_e*e/(2*m_e))( (S1.S2)/r^3 - 3*(S1.r)(S2.r)/r^5 )
    
    The choice of coord syst is arbitrary: so chosse r s.t. it lies along the z axis; 
    r = a.z_cap
    H = const * (1/a^3)( S1.S2 - 3* S1_z.S2_z )
    since, S1.S2 = 1/2(S^2 - S1^2 - S2^2 )
    also, S_z^2 = (S1_z + S2_z)^2 = S1_z^2 + S2_z^2 + 2*(S1_z.S2_z)
    S1_z.S2_z = 1/2(Sz^2 - S1_z^2 - S2_z^2)]
    H = const*(1/a^3)(1/2(S^2 - S1^2 - S2^2) - 3/2(Sz^2 - S1_z^2 - S2_z^2))

If system (here two particle) is expressed in coupled state bases; this state is an eigenfunction of H. 

In [10]:
def dipole_dipole_interaction(a, particle_1, particle_2, function_list, right_vec, left_vec ):
    h_cut = 1
    g1 = particle_1["g"]
    q1 = particle_1["q"]
    m1 = particle_1["m"]
    g2 = particle_2["g"]
    q2 = particle_2["q"]
    m2 = particle_2["m"]

    const = (g1*q1/(2*m1))*(g2*q2/(2*m2))
    
    operate = individual_spin_operation_matrix()
    S1_2 = operate["S1_2"]
    S2_2 = operate["S2_2"]
    S1z = operate["S1z"]
    S2z = operate["S2z"]
    Sz = Sz_operate(function_list)
    S_2  = S_2_operate(function_list)
    
    Sz_2 = np.square(Sz)
    S1z_2 = np.square(S1z)
    S2z_2 = np.square(S2z)
    
    H = (const/(a**3))*( (1/2)*(S_2 - S1_2 - S2_2) - (3/2)*(Sz_2 - S1z_2 - S2z_2))
    
    bra_vec = norm_column_vec(left_vec)
    ket_vec = norm_column_vec(right_vec)
    
    ene = bra_vec.T @ H @ ket_vec
    
    return np.squeeze(ene)

In [11]:
# considering a = hydrogen atom radius in atomic unit; a = 1
# g_e = 2, e = 1, h_cut = 1, m_e = 1, a = 1
# the constants, if the particle is electron
g_e = 2
q_e = 1
m_e = 1
electron_particle = {"g": g_e, "q": q_e, "m": m_e}

In [12]:
# info of proton as a particle
g_p = 5.6
q_p = -1
m_p = 1836
proton_particle = {"g": g_p, "q":q_p, "m": m_p}

In [13]:
function_list, function_dict = product_bases()
CG_matrix = CG_coeff(function_list)
coupled_states = coupled_bases(CG_matrix)

In [14]:
a = 1
particle_1 = electron_particle
particle_2 = proton_particle
ini_state = coupled_states["singlet"]
final_state = coupled_states["singlet"]

dd_interaction_energy = dipole_dipole_interaction(a,particle_1, particle_2, function_list,ini_state,final_state)
dd_interaction_energy

0.0

In [15]:
a = 1
particle = electron_particle
ini_state = coupled_states["triplet1"]
final_state = coupled_states["triplet1"]

dd_interaction_energy = dipole_dipole_interaction(a,particle_1, particle_2, function_list,ini_state,final_state)
dd_interaction_energy

0.0007625272331154684

In [16]:
a = 1
particle = electron_particle
ini_state = coupled_states["triplet_1"]
final_state = coupled_states["triplet_1"]

dd_interaction_energy = dipole_dipole_interaction(a,particle_1, particle_2, function_list,ini_state,final_state)
dd_interaction_energy

0.0007625272331154684

In [18]:
a = 1
particle = electron_particle
ini_state = coupled_states["triplet0"]
final_state = coupled_states["triplet0"]

dd_interaction_energy = dipole_dipole_interaction(a,particle_1, particle_2, function_list,ini_state,final_state)
dd_interaction_energy

-0.0015250544662309367